In [1]:
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf


from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import regex as re
import transformers
from keras import backend as K




from collections import Counter
tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))

##tensorflow-2.16.1	cudnn 8.9 cuda12.3
data=pd.read_csv('mbti_1.csv')
data.head()


2024-04-24 21:44:54.999115: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-24 21:45:13.923850: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-24 21:45:43.661762: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [2]:
def clean_text(data):
    data_length=[]
    lemmatizer=WordNetLemmatizer()
    cleaned_text=[]
    for sentence in tqdm(data.posts):
        sentence=sentence.lower()
        
        #removing links from text data
        sentence=re.sub('https?://[^\s<>"]+|www\.[^\s<>"]+',' ',sentence)
    
        #removing other symbols
        sentence=re.sub('[^0-9a-z]',' ',sentence)
    
        
        data_length.append(len(sentence.split()))
        cleaned_text.append(sentence)
    return cleaned_text

In [3]:
data.posts = clean_text(data)


  0%|          | 0/8675 [00:00<?, ?it/s]

In [4]:
#balance the dataset
print(data['type'].value_counts())
min_samples, max_samples = 200, 1000

# pre-processing (up-sampling, down-sampling, shuffle)

def resample_data(data, min_samples, max_samples):

    resampled_data = []
    for label, group in data.groupby('type'):
        n_samples = len(group)
        if n_samples < min_samples:

            resampled_group = group.sample(min_samples, replace=True, random_state=42)
        elif n_samples > max_samples:
            
            resampled_group = group.sample(max_samples, random_state=42)
        else:
            
            resampled_group = group
        resampled_data.append(resampled_group)

    
    balanced_df = pd.concat(resampled_data)

    shuffled_df = balanced_df.sample(frac=1).reset_index(drop=True)


    print(shuffled_df['type'].value_counts())

    return shuffled_df

data = resample_data(data, min_samples, max_samples)

print(data.head())


type
INFP    1832
INFJ    1470
INTP    1304
INTJ    1091
ENTP     685
ENFP     675
ISTP     337
ISFP     271
ENTJ     231
ISTJ     205
ENFJ     190
ISFJ     166
ESTP      89
ESFP      48
ESFJ      42
ESTJ      39
Name: count, dtype: int64
type
INFP    1000
INFJ    1000
INTJ    1000
INTP    1000
ENTP     685
ENFP     675
ISTP     337
ISFP     271
ENTJ     231
ISTJ     205
ESTP     200
ESFJ     200
ENFJ     200
ESTJ     200
ISFJ     200
ESFP     200
Name: count, dtype: int64
   type                                              posts
0  INFP   i guess i have a tendency to encourage people...
1  ENTP  haven t had time to think    oh  christ  now i...
2  INFJ   i ll let others judge the coherence of your p...
3  INTJ   same here  she s an enfp as well    thanks fo...
4  ENFP   eeeeeee  i touch everything when i m shopping...


In [5]:
length_col = []
for i in range(len(data)):
    length_col.append(len(data['posts'][i]))
mean_len = np.mean(np.array(length_col))
print(length_col)
print(mean_len)

[8198, 6560, 8872, 5924, 8015, 8242, 7789, 8298, 8697, 7108, 7148, 5168, 3295, 9304, 3133, 5073, 4911, 6437, 7456, 5186, 5426, 3546, 8709, 6860, 5633, 4087, 8102, 7883, 7360, 6659, 6771, 4423, 6919, 6327, 5253, 7589, 6364, 4923, 5903, 2010, 8286, 9148, 6098, 6636, 7850, 7901, 2929, 9167, 7185, 7587, 9088, 5654, 6922, 4341, 4558, 9024, 3382, 6172, 7621, 9495, 6280, 8356, 8690, 6341, 7873, 4182, 7898, 5626, 7104, 5472, 7262, 9495, 8382, 3572, 6808, 8175, 8292, 6867, 9899, 6647, 9262, 5798, 4178, 8402, 5338, 8573, 6226, 8725, 6222, 8393, 6923, 8089, 8082, 3677, 8435, 6392, 7440, 4293, 8992, 7042, 9293, 9332, 7952, 7434, 3454, 9032, 8708, 6737, 4085, 3199, 5377, 7833, 7175, 3943, 6227, 8632, 6186, 8059, 9266, 6726, 8659, 6216, 8278, 7805, 4033, 4793, 6003, 6085, 8317, 7739, 7870, 7730, 4882, 9021, 8921, 6044, 9636, 6863, 5586, 6055, 8438, 6048, 9303, 4829, 6621, 7262, 7067, 9415, 1440, 7757, 6254, 6676, 8014, 9085, 7375, 8176, 9764, 8311, 7206, 9728, 6746, 7743, 9043, 4746, 5711, 6352, 667

In [6]:
from sklearn.model_selection import train_test_split

posts = data['posts'].values
labels =  data['type'].values
train_data, test_data, train_labels, test_labels = train_test_split(posts, labels, random_state=0, test_size=0.2)

print(len(train_data))
print(len(test_data))

6083
1521


In [7]:
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences

bert_model_name = 'bert-base-uncased'

tokenizer = BertTokenizer.from_pretrained(bert_model_name, do_lower_case=True, add_special_tokens=True, return_tensors="tf")
MAX_LEN = 512
BATCH_SIZE = 128

def tokenize_sentences(sentences, tokenizer, max_seq_len = 512):
    tokenized_sentences = []

    for sentence in tqdm(sentences):
        tokenized_sentence = tokenizer.encode(
                            sentence,                  # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = max_seq_len,  # Truncate all sentences.
                            truncation=True
                    )
        
        tokenized_sentences.append(tokenized_sentence)
        
    return tokenized_sentences


types = np.unique(data.type.values)
print(types)
def get_type_index(str_array):
    return_list = []

    for i in range(len(str_array)):
        return_list.append(list(types).index(str_array[i]))

    return return_list


train_labels_categorized = get_type_index(train_labels)





def tokenize_sentences_chunking(sentences, labels, tokenizer, max_seq_len = 512, overlap = 64):
    tokenized_sentences = []
    new_labels = []
    for i, sentence in enumerate(tqdm(sentences)):
        tokenized_sentence = tokenizer.encode(
                            sentence,                  # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            
                    )
        chunk_size = max_seq_len - overlap
        chunks = [tokenized_sentence[m:m + max_seq_len] for m in range(0, len(tokenized_sentence), chunk_size)]
        overlap_labels = [labels[i] for _ in range(0, len(tokenized_sentence), chunk_size) ]
        
        tokenized_sentences += chunks
        new_labels += overlap_labels

        
    return tokenized_sentences, new_labels


train_labels_categorized = get_type_index(train_labels)
print(Counter(train_labels_categorized))




['ENFJ' 'ENFP' 'ENTJ' 'ENTP' 'ESFJ' 'ESFP' 'ESTJ' 'ESTP' 'INFJ' 'INFP'
 'INTJ' 'INTP' 'ISFJ' 'ISFP' 'ISTJ' 'ISTP']
Counter({11: 813, 8: 807, 9: 806, 10: 776, 3: 557, 1: 535, 15: 258, 13: 216, 2: 187, 14: 177, 6: 169, 5: 164, 0: 157, 12: 154, 7: 154, 4: 153})


In [8]:
from collections import Counter

print(len(train_labels))
print(Counter(train_labels))

6083
Counter({'INTP': 813, 'INFJ': 807, 'INFP': 806, 'INTJ': 776, 'ENTP': 557, 'ENFP': 535, 'ISTP': 258, 'ISFP': 216, 'ENTJ': 187, 'ISTJ': 177, 'ESTJ': 169, 'ESFP': 164, 'ENFJ': 157, 'ISFJ': 154, 'ESTP': 154, 'ESFJ': 153})


In [9]:
# chunking

train_input_ids, train_labels = tokenize_sentences_chunking(train_data,train_labels, tokenizer, MAX_LEN, overlap = 64)
train_input_ids = pad_sequences(train_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
train_labels_categorized = get_type_index(train_labels)
one_hot_train_labels = tf.keras.utils.to_categorical(train_labels_categorized, num_classes=16)
one_hot_train_labels = tf.cast(one_hot_train_labels, dtype=tf.float32)

train_dataset = tf.data.Dataset.from_tensor_slices((train_input_ids, one_hot_train_labels))
train_dataset = train_dataset.batch(BATCH_SIZE)





test_input_ids, test_labels = tokenize_sentences_chunking(test_data,test_labels, tokenizer, MAX_LEN, overlap = 64)
test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
test_labels_categorized = get_type_index(test_labels)
one_hot_test_labels = tf.keras.utils.to_categorical(test_labels_categorized, num_classes=16)
one_hot_test_labels = tf.cast(one_hot_test_labels, dtype=tf.float32)


test_dataset = tf.data.Dataset.from_tensor_slices((test_input_ids, one_hot_test_labels))
test_dataset = test_dataset.batch(BATCH_SIZE)

  0%|          | 0/6083 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1755 > 512). Running this sequence through the model will result in indexing errors
2024-04-24 21:49:56.302457: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43604 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:21:00.0, compute capability: 8.6


  0%|          | 0/1521 [00:00<?, ?it/s]

In [10]:
#SSDD dataset

vocabulary = tokenizer.get_vocab()

SSDD_dataset = pd.read_csv('Estimated_semantic_dimensions_bert_English.csv')

#SSDD_dataset.drop(SSDD_dataset['word'] == <NA>)

vocab_token = set(vocabulary.keys())

SSDD_filtered = SSDD_dataset[SSDD_dataset['word'].isin(vocab_token)]

missing_words = vocab_token.difference(set(SSDD_filtered['word']))

df_missing = pd.DataFrame({
    'word': list(missing_words),
    'Vision': [0 for _ in range(len(missing_words))],
    'Motor': [0 for _ in range(len(missing_words))],
    'Socialness': [0 for _ in range(len(missing_words))],
    'Emotion': [0 for _ in range(len(missing_words))],
    'Emotion_abs+1': [1 for _ in range(len(missing_words))],
    'Time': [0 for _ in range(len(missing_words))],
    'Space': [0 for _ in range(len(missing_words))],
    
})

SSDD_final = pd.concat([SSDD_filtered, df_missing], ignore_index=True)




SSDD_final.set_index('word', inplace=True)

SSDD_final = SSDD_final.reindex(list(vocabulary.keys()))

# SSDD_final.reset_index(inplace=True)




# SSDD_final['token_ids'] = SSDD_final['word'].apply(lambda x: vocabulary[x])

print(len(SSDD_final))

SSDD_matrix = SSDD_final.to_numpy().astype(np.float32)

print(SSDD_matrix.shape)

30522
(30522, 7)


In [11]:
word = 'apple'

# 使用tokenizer获取token ids
token_ids = tokenizer.encode(word, add_special_tokens=False)

print("Token IDs:", token_ids)
print(vocabulary[word])

Token IDs: [6207]
6207


In [12]:
# # truncating 
# one_hot_train_labels = tf.keras.utils.to_categorical(train_labels_categorized, num_classes=16)
# one_hot_train_labels = tf.cast(one_hot_train_labels, dtype=tf.float32)


# train_input_ids = tokenize_sentences(train_data, tokenizer, MAX_LEN)
# train_input_ids = pad_sequences(train_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
# train_dataset = tf.data.Dataset.from_tensor_slices((train_input_ids, one_hot_train_labels))
# train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(10000)

# one_hot_test_labels = tf.keras.utils.to_categorical(test_labels_categorized, num_classes=16)
# one_hot_test_labels = tf.cast(one_hot_test_labels, dtype=tf.float32)
# test_input_ids = tokenize_sentences(test_data, tokenizer, MAX_LEN)
# test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
# test_dataset = tf.data.Dataset.from_tensor_slices((test_input_ids, one_hot_test_labels))
# test_dataset = test_dataset.batch(BATCH_SIZE).shuffle(10000)


In [13]:

print(len(train_input_ids))


22175


In [14]:
print(one_hot_train_labels[1])

tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.], shape=(16,), dtype=float32)


In [15]:
print(train_input_ids)
print(train_dataset)

[[  101  1052  1055 ...  4895 20192 24658]
 [ 1045  3745  1037 ... 10187  1999  1996]
 [27128  2043  2017 ...  4301  2030  4784]
 ...
 [ 2057  2428  2790 ...  2008  2003  2025]
 [ 1998  2119  2024 ...  3259  1998  2031]
 [ 2241  2006  2026 ...     0     0     0]]
<_BatchDataset element_spec=(TensorSpec(shape=(None, 512), dtype=tf.int64, name=None), TensorSpec(shape=(None, 16), dtype=tf.float32, name=None))>


In [16]:
from transformers import TFBertModel

from tensorflow.keras.layers import Dense, Flatten
#@tf.keras.saving.register_keras_serializable()
class BertClassifier_mbti(tf.keras.Model):    
     def __init__(self, bert: TFBertModel, num_classes: int, SSDD_matrix = SSDD_matrix):
          super().__init__()
          self.bert = bert
          
          self.classifier = Dense(num_classes, activation='softmax')

          num_tokens, SSDD_embedding_dim = SSDD_matrix.shape

          self.SSDD_embedding_layer = tf.keras.layers.Embedding(
                              input_dim=num_tokens,    
                              output_dim=SSDD_embedding_dim, 
                              embeddings_initializer=tf.keras.initializers.Constant(SSDD_matrix),  
                              trainable=False  
                              )
          
          for layer in self.bert.layers:
               layer.trainable = True
       
     def call(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None):
          outputs = self.bert(input_ids,
                              attention_mask=attention_mask,
                              token_type_ids=token_type_ids,
                              position_ids=position_ids,
                              head_mask=head_mask)
          bert_output = outputs[0]#(none,512,1024)

          SSDD_weight = self.SSDD_embedding_layer(input_ids)#(none,512,7)

          SSDD_weight = tf.expand_dims(SSDD_weight, axis = 3)#(none,512,7,1)
          
          bert_output = tf.expand_dims(bert_output, axis = 2)#(none,512,1,1024)

          SSDD_output = SSDD_weight * bert_output # (none,512,7,1024)

          bert_output2 = tf.reshape(SSDD_output,[tf.shape(SSDD_output)[0], 512*1024*7])
          #print(bert_output2.shape)
          output = self.classifier(bert_output2)
          #print(output.shape)
          
          return output
        


def get_config(self):
        return {
            "bert": self.bert,
            "classifier": self.classifier,
            "SSDD_embedding_layer": self.SSDD_embedding_layer,
        }




In [17]:
#import tensorflow_addons as tfa
bert_layer = transformers.TFBertModel.from_pretrained('bert-large-uncased')
# for layer in bert_layer.layers:
#     layer.trainable = False
learning_rate = 2e-5
model = BertClassifier_mbti(bert_layer, 16)
model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
              metrics=['accuracy',
                              tf.keras.metrics.Precision(),
                              tf.keras.metrics.Recall(),
                              tf.keras.metrics.F1Score()]
                              )
            
    


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
import wandb
EPOCHES = 20
wandb.login()
wandb.init(project="#########", entity="########")

In [19]:

from tensorflow.keras.models import load_model
from wandb.keras import WandbMetricsLogger


#model = load_model('BERT_model.keras')  # 加载模型
#tf.config.optimizer.set_experimental_options({"disable_meta_optimizer": True})
model.fit((train_dataset), 
           
          verbose = 1, #progress report
          epochs = EPOCHES, 
          callbacks=[wandb.keras.WandbMetricsLogger()]
          )

Epoch 1/20


I0000 00:00:1714009917.681823 3490703 service.cc:145] XLA service 0x149f700176f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714009917.681850 3490703 service.cc:153]   StreamExecutor device (0): NVIDIA A40, Compute Capability 8.6
2024-04-24 21:51:59.998943: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
W0000 00:00:1714009923.091416 3490703 assert_op.cc:38] Ignoring Assert operator bert_classifier_mbti_1/tf_bert_model/bert/embeddings/assert_less/Assert/Assert
2024-04-24 21:52:06.115939: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
2024-04-24 21:52:07.493959: W external/local_xla/xla/service/gpu/triton_autotuner.cc:660] Compiling 44 configs for 2 fusions on a single thread.
I0000 00:00:1714009945.389487 3490703 device_compiler.h:188] Compiled cluster using XLA!  This line is logg

173/174 ━━━━━━━━━━━━━━━━━━━━ 1s 2s/step - accuracy: 0.1505 - f1_score: 0.1522 - loss: 14.0737 - precision: 0.1606 - recall: 0.1231

W0000 00:00:1714010229.730882 3490703 assert_op.cc:38] Ignoring Assert operator bert_classifier_mbti_1/tf_bert_model/bert/embeddings/assert_less/Assert/Assert
2024-04-24 21:57:11.911994: W external/local_xla/xla/service/gpu/triton_autotuner.cc:660] Compiling 38 configs for 2 fusions on a single thread.


174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.1508 - f1_score: 0.1527 - loss: 14.0400 - precision: 0.1610 - recall: 0.1233

wandb: ERROR Unable to log learning rate.


174/174 ━━━━━━━━━━━━━━━━━━━━ 349s 2s/step - accuracy: 0.1511 - f1_score: 0.1533 - loss: 14.0068 - precision: 0.1613 - recall: 0.1236
Epoch 2/20
174/174 ━━━━━━━━━━━━━━━━━━━━ 284s 2s/step - accuracy: 0.6502 - f1_score: 0.7393 - loss: 1.9524 - precision: 0.6987 - recall: 0.6155
Epoch 3/20
174/174 ━━━━━━━━━━━━━━━━━━━━ 322s 2s/step - accuracy: 0.8021 - f1_score: 0.8695 - loss: 1.0482 - precision: 0.8392 - recall: 0.7726
Epoch 4/20
174/174 ━━━━━━━━━━━━━━━━━━━━ 321s 2s/step - accuracy: 0.8936 - f1_score: 0.9233 - loss: 0.4260 - precision: 0.9178 - recall: 0.8686
Epoch 5/20
174/174 ━━━━━━━━━━━━━━━━━━━━ 322s 2s/step - accuracy: 0.9171 - f1_score: 0.9416 - loss: 0.4913 - precision: 0.9327 - recall: 0.8956
Epoch 6/20
174/174 ━━━━━━━━━━━━━━━━━━━━ 322s 2s/step - accuracy: 0.8914 - f1_score: 0.9223 - loss: 0.6685 - precision: 0.9042 - recall: 0.8718
Epoch 7/20
174/174 ━━━━━━━━━━━━━━━━━━━━ 322s 2s/step - accuracy: 0.9354 - f1_score: 0.9529 - loss: 0.4181 - precision: 0.9441 - recall: 0.9202
Epoch 8/2

In [20]:
model.save('BERT_model.keras') 


In [ ]:
loss, accuracy, f1_score, precision, recall = model.evaluate(test_dataset, verbose = 1)

In [ ]:

result_dict = {
    'loss' : loss,
    'accuracy' : accuracy,
    'f1_score' : f1_score,
    'precision' : precision,
    'recall' : recall
}
df = pd.DataFrame(result_dict)
print(df)
df.to_csv('result.csv', index=False)
#df.head()  # >:DataFrame